# Homework 5 - Exploring Neural Network Architectures for Advancing and Predicting Chaotic Dynamics in the Lorenz System
# Github: https://github.com/idane2309 | Ishan Dane

### 1a. Train a NN to advance the solution from t to t + ∆t for ρ = 10, 28 and 40. 

In [68]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [289]:
# Train and test values for rho
rho_train_values = [10, 28, 40]
rho_test_values = [17, 35]

def get_lorenz_deriv(rho):
    # Initialize parameters
    dt = 0.01
    T = 8
    t = np.arange(0,T+dt,dt)
    beta = 8/3
    sigma = 10

    # Initialize input and output arrays for rho value
    nn_input = np.zeros((100 * (len(t) - 1), 3))
    nn_output = np.zeros_like(nn_input)

    # Define Lorenz system
    def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
        x, y, z = x_y_z
        return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]
    
    # Solve Lorenz system for rho value
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t)
                    for x0_j in x0])

    for j in range(100):
        nn_input[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t[j,:-1,:]
        nn_output[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t[j,1:,:]
    
    return nn_input, nn_output

def create_train_data(rho_train_values):
    # Initialize input and output arrays
    nn_input = np.zeros((0, 3))
    nn_output = np.zeros_like(nn_input)

    # Get training data for each rho value
    for rho in rho_train_values:
        nn_input_rho, nn_output_rho = get_lorenz_deriv(rho)
        nn_input = np.concatenate((nn_input, nn_input_rho))
        nn_output = np.concatenate((nn_output, nn_output_rho))
    
    # Convert to torch tensors
    nn_input = torch.from_numpy(nn_input).float()
    nn_output = torch.from_numpy(nn_output).float()

    return nn_input, nn_output

def create_test_data(rho):
    nn_test_input, nn_test_output = get_lorenz_deriv(rho)
    nn_test_input = torch.from_numpy(nn_test_input).float()
    nn_test_output = torch.from_numpy(nn_test_output).float()

    return nn_test_input, nn_test_output


   


In [313]:
# Training
# Define activation functions
def logsig(x):
    return 1 / (1 + torch.exp(-x))

def radbas(x):
    return torch.exp(-torch.pow(x, 2))

def purelin(x):
    return x

# Define the model
class FFNN(nn.Module):
    def __init__(self):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(in_features=3, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=10)
        self.fc3 = nn.Linear(in_features=10, out_features=3)

    def forward(self, x):
        x = logsig(self.fc1(x))
        x = radbas(self.fc2(x))
        x = purelin(self.fc3(x))
        return x

# Create Feed Forward NN model instance
ffnn = FFNN()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(ffnn.parameters(), lr=0.01)

# Train the model
ffnn_input_train, ffnn_output_train = create_train_data(rho_train_values)

for epoch in range(50):
    optimizer.zero_grad()
    ffnn_output_pred = ffnn(ffnn_input_train)
    loss = criterion(ffnn_output_pred, ffnn_output_train)
    loss.backward()
    optimizer.step()
    print('Epoch: ', epoch, 'Loss: ', loss.item())




Epoch:  0 Loss:  290.7338562011719
Epoch:  1 Loss:  268.8385925292969
Epoch:  2 Loss:  248.95135498046875
Epoch:  3 Loss:  231.1382293701172
Epoch:  4 Loss:  215.1306610107422
Epoch:  5 Loss:  200.9007110595703
Epoch:  6 Loss:  188.52249145507812
Epoch:  7 Loss:  177.84596252441406
Epoch:  8 Loss:  168.62496948242188
Epoch:  9 Loss:  160.67105102539062
Epoch:  10 Loss:  153.8308563232422
Epoch:  11 Loss:  147.9592742919922
Epoch:  12 Loss:  142.92112731933594
Epoch:  13 Loss:  138.59544372558594
Epoch:  14 Loss:  134.87747192382812
Epoch:  15 Loss:  131.67799377441406
Epoch:  16 Loss:  128.92086791992188
Epoch:  17 Loss:  126.54083251953125
Epoch:  18 Loss:  124.48184204101562
Epoch:  19 Loss:  122.69554138183594
Epoch:  20 Loss:  121.14019775390625
Epoch:  21 Loss:  119.7796630859375
Epoch:  22 Loss:  118.5826644897461
Epoch:  23 Loss:  117.52207946777344
Epoch:  24 Loss:  116.57438659667969
Epoch:  25 Loss:  115.71939086914062
Epoch:  26 Loss:  114.93975830078125
Epoch:  27 Loss:  11

### 1b. Now see how well your NN works for future state prediction for ρ = 17 and ρ = 35

In [314]:

# Testing
# Test the model
for rho in rho_test_values:
    ffnn_input_test, ffnn_output_test = create_test_data(rho)
    ffnn_output_pred = ffnn(ffnn_input_test)
    loss = criterion(ffnn_output_pred, ffnn_output_test)
    print('Loss for rho = ', rho, ': ', loss.item())


Loss for rho =  17 :  51.017696380615234
Loss for rho =  35 :  119.07109832763672


### 2. Compare feed-forward, LSTM, RNN and Echo State Networks for forecasting the dynamics.

#### Long Short-Term Memory (LSTM)

In [294]:
# Create LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size=10, output_size=3):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_layer_size)
        c0 = torch.zeros(1, x.size(0), self.hidden_layer_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.linear(out[:, -1, :])
        return out

In [315]:
# Create model instance
lstm = LSTM()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(lstm.parameters(), lr=0.01, momentum=0.9)

# Get training data
lstm_input_train, lstm_output_train = create_train_data(rho_train_values)

# Reshape the input data for LSTM
lstm_input_train = lstm_input_train.reshape(lstm_input_train.shape[0], 1, lstm_input_train.shape[1])

# Train the model
for epoch in range(50):
    optimizer.zero_grad()
    lstm_output_pred = lstm(lstm_input_train)
    loss = criterion(lstm_output_pred, lstm_output_train)
    loss.backward()
    optimizer.step()
    print('Epoch: ', epoch, 'Loss: ', loss.item())


Epoch:  0 Loss:  298.1119689941406
Epoch:  1 Loss:  290.7731018066406
Epoch:  2 Loss:  281.8709716796875
Epoch:  3 Loss:  272.04144287109375
Epoch:  4 Loss:  258.02947998046875
Epoch:  5 Loss:  238.87533569335938
Epoch:  6 Loss:  216.79537963867188
Epoch:  7 Loss:  193.85147094726562
Epoch:  8 Loss:  172.54293823242188
Epoch:  9 Loss:  153.50540161132812
Epoch:  10 Loss:  137.70582580566406
Epoch:  11 Loss:  125.08660888671875
Epoch:  12 Loss:  115.65365600585938
Epoch:  13 Loss:  108.48930358886719
Epoch:  14 Loss:  104.91077423095703
Epoch:  15 Loss:  102.40277862548828
Epoch:  16 Loss:  101.83796691894531
Epoch:  17 Loss:  103.71575927734375
Epoch:  18 Loss:  103.82256317138672
Epoch:  19 Loss:  102.56806945800781
Epoch:  20 Loss:  96.78571319580078
Epoch:  21 Loss:  94.56050109863281
Epoch:  22 Loss:  85.36528015136719
Epoch:  23 Loss:  78.07530975341797
Epoch:  24 Loss:  83.62678527832031
Epoch:  25 Loss:  85.9566879272461
Epoch:  26 Loss:  83.40065002441406
Epoch:  27 Loss:  79.0

In [316]:
# Test the model
for rho in rho_test_values:
    lstm_input_test, lstm_output_test = create_test_data(rho)
    lstm_input_test = lstm_input_test.reshape(lstm_input_test.shape[0], 1, lstm_input_test.shape[1])
    lstm_output_pred = lstm(lstm_input_test)
    loss = criterion(lstm_output_pred, lstm_output_test)
    print('Loss for rho = ', rho, ': ', loss.item())

Loss for rho =  17 :  34.123451232910156
Loss for rho =  35 :  55.88348388671875


#### RNN

In [128]:
# RNN 
class RNN(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size=10, output_size=3):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.rnn = nn.RNN(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_layer_size)
        out, _ = self.rnn(x, h0)
        out = self.linear(out[:, -1, :])
        return out

In [298]:
# Create model instance
rnn = RNN()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(rnn.parameters(), lr=0.01, momentum=0.9)

# Get training data
rnn_input_train, rnn_output_train = create_train_data(rho_train_values)

# Reshape the input data for RNN
rnn_input_train = rnn_input_train.reshape(rnn_input_train.shape[0], 1, rnn_input_train.shape[1])

# Train the model
for epoch in range(50):
    optimizer.zero_grad()
    rnn_output_pred = rnn(rnn_input_train)
    loss = criterion(rnn_output_pred, rnn_output_train)
    loss.backward()
    optimizer.step()
    print('Epoch: ', epoch, 'Loss: ', loss.item())

Epoch:  0 Loss:  285.5103454589844
Epoch:  1 Loss:  264.4383850097656
Epoch:  2 Loss:  235.49020385742188
Epoch:  3 Loss:  198.60562133789062
Epoch:  4 Loss:  160.79833984375
Epoch:  5 Loss:  130.48828125
Epoch:  6 Loss:  112.53905487060547
Epoch:  7 Loss:  104.56169891357422
Epoch:  8 Loss:  141.25018310546875
Epoch:  9 Loss:  99.77456665039062
Epoch:  10 Loss:  103.42486572265625
Epoch:  11 Loss:  108.13023376464844
Epoch:  12 Loss:  108.3995132446289
Epoch:  13 Loss:  102.33721160888672
Epoch:  14 Loss:  101.24153137207031
Epoch:  15 Loss:  98.57857513427734
Epoch:  16 Loss:  90.08848571777344
Epoch:  17 Loss:  95.14173126220703
Epoch:  18 Loss:  93.9048080444336
Epoch:  19 Loss:  90.76856994628906
Epoch:  20 Loss:  86.56181335449219
Epoch:  21 Loss:  81.3731918334961
Epoch:  22 Loss:  76.72736358642578
Epoch:  23 Loss:  73.02940368652344
Epoch:  24 Loss:  65.07286071777344
Epoch:  25 Loss:  70.8486557006836
Epoch:  26 Loss:  70.10623931884766
Epoch:  27 Loss:  64.13890075683594
Epo

In [299]:
# Test the model
for rho in rho_test_values:
    rnn_input_test, rnn_output_test = create_test_data(rho)
    rnn_input_test = rnn_input_test.reshape(rnn_input_test.shape[0], 1, rnn_input_test.shape[1])
    rnn_output_pred = rnn(rnn_input_test)
    loss = criterion(rnn_output_pred, rnn_output_test)
    print('Loss for rho = ', rho, ': ', loss.item())

Loss for rho =  17 :  43.22047805786133
Loss for rho =  35 :  55.39307403564453


#### Echo State Network (ESN)

In [217]:
# Echo State Network for Lorenz System
class Reservoir(nn.Module):
  def __init__(self, hidden_dim, connectivity):
    super().__init__()
    
    self.Wx = self.sparse_matrix(hidden_dim, connectivity)
    self.Wh = self.sparse_matrix(hidden_dim, connectivity)
    self.Uh = self.sparse_matrix(hidden_dim, connectivity)
    self.act = nn.Tanh()

  def sparse_matrix(self, m, p):
    mask_distribution = torch.distributions.Bernoulli(p)
    S = torch.randn((m, m))
    mask = mask_distribution.sample(S.shape)
    S = (S*mask).to_sparse()
    return S

  def forward(self, x, h):
    h = self.act(torch.sparse.mm(self.Uh, h.T).T +
                 torch.sparse.mm(self.Wh, x.T).T)
    y = self.act(torch.sparse.mm(self.Wx, h.T).T)

    return y, h
     
class EchoState(nn.Module):
  def __init__(self, in_dim, out_dim, reservoir_dim, connectivity):
    super().__init__()

    self.reservoir_dim = reservoir_dim
    self.input_to_reservoir = nn.Linear(in_dim, reservoir_dim)
    self.input_to_reservoir.requires_grad_(False)

    self.reservoir = Reservoir(reservoir_dim, connectivity)
    self.readout = nn.Linear(reservoir_dim, out_dim)
  
  def forward(self, x):
    reservoir_in = self.input_to_reservoir(x)
    h = torch.ones(x.size(0), self.reservoir_dim)
    reservoirs = []
    for i in range(x.size(1)):
      out, h = self.reservoir(reservoir_in[:, i, :], h)
      reservoirs.append(out.unsqueeze(1))
    reservoirs = torch.cat(reservoirs, dim=1)
    outputs = self.readout(reservoirs)
    return outputs

In [317]:
# Create model instance
esn = EchoState(3, 3, 50, 0.1)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(esn.parameters(), lr=0.01, momentum=0.9)

# Get training data
esn_input_train, esn_output_train = create_train_data(rho_train_values)

# Train the model
for epoch in range(50):
    optimizer.zero_grad()
    outputs = esn(esn_input_train.view(1, -1, 3))
    loss = criterion(outputs, esn_output_train.view(1, -1, 3))
    loss.backward()
    optimizer.step()
    print('Epoch: ', epoch, 'Loss: ', loss.item())

Epoch:  0 Loss:  299.104248046875
Epoch:  1 Loss:  245.12301635742188
Epoch:  2 Loss:  167.2060546875
Epoch:  3 Loss:  100.9609146118164
Epoch:  4 Loss:  69.63018035888672
Epoch:  5 Loss:  75.55786895751953
Epoch:  6 Loss:  103.31529235839844
Epoch:  7 Loss:  130.7093048095703
Epoch:  8 Loss:  140.83364868164062
Epoch:  9 Loss:  129.14903259277344
Epoch:  10 Loss:  103.14781951904297
Epoch:  11 Loss:  76.22244262695312
Epoch:  12 Loss:  59.85285568237305
Epoch:  13 Loss:  58.24943923950195
Epoch:  14 Loss:  67.59815979003906
Epoch:  15 Loss:  79.40253448486328
Epoch:  16 Loss:  85.5162353515625
Epoch:  17 Loss:  82.08946990966797
Epoch:  18 Loss:  70.70177459716797
Epoch:  19 Loss:  56.64531707763672
Epoch:  20 Loss:  45.714744567871094
Epoch:  21 Loss:  41.331932067871094
Epoch:  22 Loss:  43.333168029785156
Epoch:  23 Loss:  48.695655822753906
Epoch:  24 Loss:  53.49345016479492
Epoch:  25 Loss:  54.9140739440918
Epoch:  26 Loss:  52.35865020751953
Epoch:  27 Loss:  47.27476501464844

In [320]:
# Test the model
for rho in rho_test_values:
    esn_input_test, esn_output_test = create_test_data(rho)
    outputs = esn(esn_input_test.view(1, -1, 3))
    loss = criterion(outputs, esn_output_test.view(1, -1, 3))
    print('Loss for rho = ', rho, ': ', loss.item())


Loss for rho =  17 :  20.919633865356445
Loss for rho =  35 :  38.054019927978516
